In [1]:
import numpy as np
import pandas as pd
import pyodbc
from datetime import datetime

In [2]:
server = '============='
database = '==========='
username = '================='
driver = '{ODBC Driver 17 for SQL Server}'

cnxn = pyodbc.connect('DRIVER=' + driver +
                      ';SERVER=' + server +
                      ';DATABASE=' + database +
                      ';UID=' + username +
                      ';Trusted_Connection=yes')
cursor = cnxn.cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC Driver 17 for SQL Server]Named Pipes Provider: Could not open a connection to SQL Server [53].  (53) (SQLDriverConnect); [08001] [Microsoft][ODBC Driver 17 for SQL Server]Login timeout expired (0); [08001] [Microsoft][ODBC Driver 17 for SQL Server]A network-related or instance-specific error has occurred while establishing a connection to SQL Server. Server is not found or not accessible. Check if instance name is correct and if SQL Server is configured to allow remote connections. For more information see SQL Server Books Online. (53)')

In [ ]:
# get source dataframes
SQL_Query = pd.read_sql_query(
    "SELECT * FROM [RiskDB].[bonds].[Obligacje]", cnxn)
obligacje = pd.DataFrame(SQL_Query)

SQL_Query = pd.read_sql_query(
    "SELECT * FROM [RiskDB].[bonds].[pozycja] order by IdObligacje asc", cnxn)
pozycja = pd.DataFrame(SQL_Query)

SQL_Query = pd.read_sql_query(
    "SELECT * FROM [RiskDB].[bonds].[odsetki] order by IdObligacje, OkresDo asc", cnxn)
odsetki = pd.DataFrame(SQL_Query)

In [ ]:
# def get payments per period
def get_payments_per_period(id):
    # designed for constant coupon period !!!

    lista_okresow = []
    for index, row in obligacje.iterrows():
        if row['id'] == id:
            data_wykupu = obligacje['DataWykupu'].iloc[index]
            # ISIN = obligacje['ISIN'].iloc[index]

    # get earliest OkresOd
    for index, row in odsetki.iterrows():
        if row['idObligacje'] == id:
            lista_okresow.append(row['OkresOd'])
    # if no coupon in odsetki
    if len(lista_okresow) == 0:
        Id = id
        wina_Tuska = 'błąd: Brak zapisu w tabeli ODSETKI dla wskazanej obligacji: idObligacje: {}.'.format(Id)
        print(wina_Tuska)

        return None

    try:
        OkresOd = min(lista_okresow)

        # calculate payments count
        for index, row in odsetki.iterrows():
            if row['idObligacje'] == id and row['OkresOd'] == OkresOd:
                # get OkresDo for coupon
                OkresDo = row['OkresDo']

                # calculate coupon period in months
                years = OkresDo.year - OkresOd.year
                years_to_months = years * 12
                months = abs(OkresDo.month - OkresOd.month)

                coupon_months_sum = years_to_months + months

                # calculate whole bond period in months
                whole_period_bond_years = data_wykupu.year - OkresOd.year
                whole_period_bond_years_to_months = whole_period_bond_years * 12
                whole_period_bond_months = abs(data_wykupu.month - OkresOd.month)

                whole_period_bond_months_sum = whole_period_bond_years_to_months + whole_period_bond_months

                # payments per bond period
                payments_count = whole_period_bond_months_sum / coupon_months_sum
                payments_count = int(payments_count)

                return payments_count

                break

    except:
        return None


# def calculate TERMS to maturity
def terms_to_maturity(payments_c):
    terms = range(1, payments_c + 1)
    terms_list = list(terms)
    terms_list_sorted = sorted(terms_list, reverse=True)
    return terms_list_sorted


# def get face value
def get_face_value(index):
    FaceValue = obligacje['Nominal'].iloc[index]
    return FaceValue


# def get current price
def current_price(id):
    strptime_date = datetime.strptime(date_insert, '%Y%m%d').date()
    dates = []

    for index, row in pozycja.iterrows():
        if row['idObligacje'] == id:
            dates.append(row['Data'])

    # if no row 'wycena' for particular bond in POZYCJA:
    if len(dates) == 0:
        for index, row in obligacje.iterrows():
            if row['id'] == id:
                ISIN = row['ISIN']
                Id = row['id']
                blad = 'błąd: Brak wyceny w tabeli POZYCJA dla wskazanej obligacji: idObligacje: {}, ISIN: {}.'.format(
                    Id, ISIN)
        print(blad)
        # return blad

    # 'wycena' exists
    else:
        # get actual 'wycena'
        dates.sort()
        past_wycena = []
        for element in dates:
            if strptime_date >= element:
                past_wycena.append(element)

        # if no value 'wycena' for particular date
        if len(past_wycena) == 0:
            blad1 = 'błąd: Brak wyceny dla podanej daty: {}. Wprowadź późniejszą datę.'.format(strptime_date)
            print(blad1)
            # return blad1

        # value 'wycena' exists
        else:
            past_wycena.sort(reverse=True)
            data_wyceny = past_wycena[0]
            for index, row in pozycja.iterrows():
                if row['Data'] == data_wyceny and row['idObligacje'] == id:
                    wycena = row['Wycena'] / row['Liczba']
            return wycena


# def get coupon rates
def get_coupon_rates(id):
    try:
        coupon_rates_list = []
        for index, row in odsetki.iterrows():
            if row['idObligacje'] == id:
                coupon_rates_list.append(row['Odsetki'])
        return coupon_rates_list
    except:
        return None


# def YTM (yield to maturity)
def YTM_constant_coupon(terms_to_m, face_value, curr_price, coupon_rate):
    try:
        ytm = [(coupon_rate[i] + ((face_value - curr_price) / (terms_to_m[i]))) / ((face_value + curr_price) / 2)
               for i in range(len(coupon_rate))]
        return ytm
    except:
        return None


# def YTM for mutable coupon (get latest coupon)
def YTM_mutable_coupon(terms_to_m, face_value, curr_price, coupon_rate):
    try:
        ytm = [(coupon_rate[-1] + ((face_value - curr_price) / (terms_to_m[i]))) / ((face_value + curr_price) / 2)
               for i in range(len(terms_to_m))]
        return ytm
    except:
        return None


def calculate_duration_constant_coupon(coupon_rate, YTM, curr_price, face_value):
    try:
        mac_dur = np.sum([coupon_rate[i] * (i + 1) / np.power(1 + YTM[i], i + 1)
                          for i in range(len(coupon_rate))])
        # final year
        power = len(coupon_rate)
        mac_dur = mac_dur - (coupon_rate[-1] / (np.power(1 + YTM[-1], power))) + \
                  ((coupon_rate[-1] + face_value) / (np.power(1 + YTM[-1], power)))

        mac_dur = mac_dur / curr_price
        # mod_dur = mac_dur / (1 + ytm / no_coupons)
        return mac_dur

    except:
        return None


def calculate_duration_mutable_coupon(coupon_rate, YTM, curr_price, face_value, terms_to_m):
    # for last coupon:
    coupon_rate = coupon_rate[-1]

    try:
        mac_dur = np.sum([coupon_rate * (i + 1) / np.power(1 + YTM[i], i + 1)
                          for i in range(len(terms_to_m))])
        # final year
        power = len(terms_to_m)
        mac_dur = mac_dur - (coupon_rate / (np.power(1 + YTM[-1], power))) + \
                  ((coupon_rate + face_value) / (np.power(1 + YTM[-1], power)))

        mac_dur = mac_dur / curr_price
        # mod_dur = mac_dur / (1 + ytm / no_coupons)
        return mac_dur

    except:
        return None


In [ ]:
# create data frame list
collect_dataframes = []

In [ ]:
# calculate duration
# type date of calculation
date_insert = '20190820'

for index, row in obligacje.iterrows():
    values = []

    # get 'id' from 'obligacje' dataframe
    idObligacje = obligacje['id'].iloc[index]
    values.append(idObligacje)

    # get payments per period
    payments_per_period = get_payments_per_period(idObligacje)
    values.append(payments_per_period)

    # calculate terms to maturity
    terms_to_mat = terms_to_maturity(get_payments_per_period(idObligacje))
    values.append(terms_to_mat)

    # get face value
    FaceValue = get_face_value(index)
    values.append(FaceValue)

    # get current price
    current_pr = current_price(idObligacje)
    values.append(current_pr)

    # get coupon rates
    coupon_rates = get_coupon_rates(idObligacje)
    values.append(coupon_rates)

    # calculate duration

    # for constant coupon
    if row['StalyKupon'] == 1:
        # calculate Yield To Maturity (YTM)
        YieldToMaturity = YTM_constant_coupon(terms_to_m=terms_to_mat,
                                              face_value=FaceValue,
                                              curr_price=current_pr,
                                              coupon_rate=coupon_rates)
        values.append(YieldToMaturity)

        m_duration = calculate_duration_constant_coupon(coupon_rate=coupon_rates,
                                                        YTM=YieldToMaturity,
                                                        curr_price=current_pr,
                                                        face_value=FaceValue)
    else:
        # For mutable coupon
        YieldToMaturity = YTM_mutable_coupon(terms_to_m=terms_to_mat,
                                             face_value=FaceValue,
                                             curr_price=current_pr,
                                             coupon_rate=coupon_rates)
        values.append(YieldToMaturity)

        m_duration = calculate_duration_mutable_coupon(coupon_rate=coupon_rates,
                                                       YTM=YieldToMaturity,
                                                       curr_price=current_pr,
                                                       face_value=FaceValue,
                                                       terms_to_m=terms_to_mat)

    values.append(m_duration)
#     print(values)

    df = pd.DataFrame({'idObligacje' : values[0],
                       'PaymentPerPeriod' : values[1],
                       'TermsToMaturity' : [values[2]],
                       'FaceValue' : values[3],
                       'CurrentPrice' : values[4],
                       'CouponRate' : [values[5]],
                       'YTM' : [values[6]],
                       'Duration' : values[7],
                       'Date_calc' : date_insert})

    # append df into list
    collect_dataframes.append(df)

In [ ]:
# create a final data frame
duration_mac = pd.concat(collect_dataframes)

duration_mac = duration_mac.reset_index(drop=True)
duration_mac